In [ ]:
import sys
sys.path.append('../')
%load_ext autotime
import numpy as np
import models.rydberg_calcs as ryd
from scipy.constants import c as C_c
import matplotlib.pyplot as plt

VCSELs are fundamentally single-mode lasers, due to their large cavity FSR, and typically emit in a single transverse mode (TEM00). They can be modulated directly at very high frequencies, and manufactured in very large and closely-spaced arrays with individually addressable sites. 

The gain section of VCSELs is very short, which fundamentally limits their total output power to only a few milliwatts. Let's explore what Rabi frequencies are achievable with few milliwatt powers on the most advantageous transitions in Cs and Rb. 

In [ ]:
laserWaist = 25e-6

We can possibly juice out a pit more Rabi frequency using very small waists with a microlens array.

In [ ]:
transition_7P32 = ryd.OpticalTransition(n1=6, l1=0, j1=0.5, mj1=0.5, f1=4, mf1=4, q=1, 
                                        n2=7, l2=1, j2=1.5, mj2=1.5, f2=5, mf2=5, laserWaist=laserWaist)
print(transition_7P32.matrixElement)
transition_7P12 = ryd.OpticalTransition(n1=6, l1=0, j1=0.5, mj1=0.5, f1=4, mf1=4, q=-1, 
                                        n2=7, l2=1, j2=1.5, mj2=1.5, f2=3, mf2=3, laserWaist=laserWaist)
print(transition_7P12.matrixElement)
transition_6P32 = ryd.OpticalTransition(n1=6, l1=0, j1=0.5, mj1=0.5, f1=4, mf1=4, q=1, 
                                        n2=6, l2=1, j2=1.5, mj2=1.5, f2=5, mf2=5, laserWaist=laserWaist)
print(transition_6P32.matrixElement)

Ground state 6S to 6P3/2 at 852nm is of course a very strong transition, so that's advantageous. What states then connect us up to the Rydberg states?

In [ ]:
transition_6P32_7S12 = ryd.OpticalTransition(n1=6, l1=1, j1=1.5, mj1=1.5, f1=5, mf1=5, q=-1, 
                                        n2=7, l2=0, j2=0.5, mj2=0.5, f2=4, mf2=4, laserWaist=laserWaist)
print(C_c/transition_6P32_7S12.get_transition_freq()*1e9, "nm")
print(transition_6P32_7S12.matrixElement)


6P3/2 to 7S1/2 is also a very strong transition, at 1470nm (good for diode lasers). How does the coupling to Rabi states compare to 7P3/2 to the Rydberg states?

In [ ]:
transition_7S12_40P32 = ryd.OpticalTransition(n1=7, l1=0, j1=0.5, mj1=0.5, f1=4, mf1=4, q=1, 
                                        n2=40, l2=1, j2=1.5, mj2=1.5, f2=5, mf2=5, laserWaist=laserWaist)
print(C_c/transition_7S12_40P32.get_transition_freq()*1e9, "nm")
print(transition_7S12_40P32.matrixElement)
transition_7P32_40S12 = ryd.OpticalTransition(n1=7, l1=1, j1=1.5, mj1=1.5, f1=5, mf1=5, q=-1, 
                                        n2=40, l2=0, j2=0.5, mj2=0.5, f2=4, mf2=4, laserWaist=laserWaist)
print(C_c/transition_7P32_40S12.get_transition_freq()*1e9, "nm")
print(transition_7P32_40S12.matrixElement)

In [ ]:
# Define the range of n2 values to plot
n2_values = range(20, 61)

# Lists to store matrix elements for both transitions
matrix_elements_7S12_nP32 = []
matrix_elements_7P32_nS12 = []
wavelengths_7S12_nP32 = []
wavelengths_7P32_nS12 = []

# Calculate matrix elements for each n2 value
for n2 in n2_values:
    # First transition: 7S1/2 to nP3/2
    transition_7S12_nP32 = ryd.OpticalTransition(
        n1=7, l1=0, j1=0.5, mj1=0.5, f1=4, mf1=4, q=1,
        n2=n2, l2=1, j2=1.5, mj2=1.5, f2=5, mf2=5, laserWaist=laserWaist
    )
    matrix_elements_7S12_nP32.append(transition_7S12_nP32.matrixElement)
    wavelengths_7S12_nP32.append(C_c/transition_7S12_nP32.get_transition_freq()*1e9)
    
    # Second transition: 7P3/2 to nS1/2
    transition_7P32_nS12 = ryd.OpticalTransition(
        n1=7, l1=1, j1=1.5, mj1=1.5, f1=5, mf1=5, q=-1,
        n2=n2, l2=0, j2=0.5, mj2=0.5, f2=4, mf2=4, laserWaist=laserWaist
    )
    matrix_elements_7P32_nS12.append(transition_7P32_nS12.matrixElement)
    wavelengths_7P32_nS12.append(C_c/transition_7P32_nS12.get_transition_freq()*1e9)

# Create the figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot matrix elements vs n2
ax1.plot(n2_values, np.abs(matrix_elements_7S12_nP32), 'o-', label='7S1/2 → nP3/2')
ax1.plot(n2_values, np.abs(matrix_elements_7P32_nS12), 's-', label='7P3/2 → nS1/2')
ax1.set_xlabel('Principal Quantum Number (n2)')
ax1.set_ylabel('abs(Matrix Element)')
ax1.set_title('Matrix Element vs Principal Quantum Number')
ax1.legend()
ax1.grid(True)

# Plot wavelengths vs n2
ax2.plot(n2_values, wavelengths_7S12_nP32, 'o-', label='7S1/2 → nP3/2')
ax2.plot(n2_values, wavelengths_7P32_nS12, 's-', label='7P3/2 → nS1/2')
ax2.set_xlabel('Principal Quantum Number (n2)')
ax2.set_ylabel('Wavelength (nm)')
ax2.set_title('Transition Wavelength vs Principal Quantum Number')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

It appears that the 7P3/2 to nS1/2 transition is always stronger than the 7S1/2 to nP3/2 transition, but only by a factor of ~2.8, not that much. The wavelength for 7P3/2 to nS1/2 is favorable, around 780nm. 

How do pulse programs in a 4-level system achieve optimal population inversion? This is a question we should address via an optimization program.

Are there any advantages to using Rb instead?